<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [ ]:
! pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import psycopg2

In [ ]:
import requests  # для парсинга в юнитах 5 и 7
from bs4 import BeautifulSoup # для парсинга в юнитах 5 и 7
import plotly.express as px # для построения графика в юните 7

In [ ]:
# Вместо psycopg2 можно было бы использовать sqlalchemy вот так:
# from sqlalchemy import create_engine
# connection = create_engine(f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')
# df = pd.read_sql_query(sqlalchemy.text(query), conn_str) # метод text - для корректного восприятия символа %

# Однако после ее обновления до версии 2.0 возникает ошибка
# 'OptionEngine' object has no attribute 'execute', о которой сказано
# здесь: https://github.com/pandas-dev/pandas/issues/51015
# В надежде на скорое исправление ситуации прибегнем к отключению
# сообщений о несовместимости pandas и SQLAlchemy:

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python

# В последствии - нужно удалить - # Параметры подключения к базе данных удалены в целях безопасности

DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [ ]:
# Создаем подключение к базе данных
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

## Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies).

In [ ]:
# текст запроса
query_3_1 = f'''SELECT
                    COUNT(id) AS total_vacancies
                FROM public.vacancies
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
print('Общее количество вакансий в базе:', df.iloc[0,0])

Общее количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers).

In [ ]:
# текст запроса
query_3_2 = f'''SELECT
                    COUNT(id) AS total_employers
                FROM employers
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
print('Общее количество работодателей в базе:', df.iloc[0,0])

Общее количество работодателей в базе: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
# текст запроса
query_3_3 = f'''SELECT
                    COUNT(id) AS total_areas
                FROM areas
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
print('Общее количество регионов в базе:', df.iloc[0,0])

Общее количество регионов в базе: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
# текст запроса
query_3_4 = f"""SELECT
                    COUNT(id) AS total_industries
                FROM industries
             """

In [ ]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
print('Общее количество сфер деятельности в базе:', df.iloc[0,0])

Общее количество сфер деятельности в базе: 294


***

***Bыводы по предварительному анализу данных***

1. В базе данных содержится 49 тыс. вакансий в 23,5 тыс. компаний.

2. География работодателей представлена 1362 регионами (городами).

4. Сферы деятельности, в которых предлагаются вакансии, насчитывают почти 300 различных видов.

## Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса
query_4_1 = f'''SELECT
                    a.name AS area,
                    COUNT(v.id) AS cnt
                FROM areas AS a
                JOIN vacancies AS v ON a.id = v.area_id
                GROUP BY a.name
                ORDER BY COUNT(v.id) DESC
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
display('Количество вакансий в каждом регионе:', df)

'Количество вакансий в каждом регионе:'

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса
query_4_2 = f'''SELECT
                    COUNT(id)
                FROM vacancies
                WHERE salary_from IS NOT NULl
                    OR salary_to IS NOT NULl
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
print('Количество вакансий, у которых заполнено хотя бы одно из двух полей с зарплатой:', df.iloc[0,0])

Количество вакансий, у которых заполнено хотя бы одно из двух полей с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
# текст запроса
query_4_3 = f'''SELECT
                    ROUND(AVG(salary_from), 0)::int AS avg_salary_from,
                    ROUND(AVG(salary_to), 0)::int AS avg_salary_to
                FROM vacancies
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
print(f'Средние значения для зарплатной вилки (в руб.): нижнее - {df.iloc[0,0]}, верхнее - {df.iloc[0,1]}.')

Средние значения для зарплатной вилки (в руб.): нижнее - 71065, верхнее - 110537.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика `(schedule)` и типа трудоустройства `(employment)`, используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса
query_4_4 = f'''SELECT
                    schedule,
                    employment,
                    COUNT(id) AS num_of_vacancies
                FROM vacancies
                GROUP BY schedule, employment
                ORDER BY 3 DESC
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
display('Количество вакансий для различных сочетаний типов графика и трудоустройства:', df)

'Количество вакансий для различных сочетаний типов графика и трудоустройства:'

,schedule,employment,num_of_vacancies
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [ ]:
# текст запроса
query_4_5 = f'''SELECT
                    experience,
                    COUNT(id) AS num_of_vacancies
                FROM vacancies
                WHERE experience IS NOT NULL
                GROUP BY experience
                ORDER BY COUNT(id) ASC
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
display('Количество вакансий для различного опыта работы:', df)

'Количество вакансий для различного опыта работы:'

,experience,num_of_vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

***Выводы по детальному анализу данных:***  

1.  Тройку городов по количеству представленных вакансий составляют: Москва, Санкт-Петербург и Минск. При этом количество вакансий в одной Москве (5333) превышает суммарное их количество в двух других городах из тройки (2851 и 2112, соответственно). Кроме этого, Москва и Питер заметно отрываются от других городов по количеству вакансий, а вот начиная с Минска и дальше вниз рейтинга разница между городами не столь драматична. Сравните: Минск (2112) - Новосибирск (2006) - Алматы (1892) и т.д.  

2. Примерно в половине вакансий (24 тыс. из 49 тыс.) работодатели указали хотя бы одно из двух значений зарплатной вилки.  

3. При этом средняя минимальная заработная плата составляет 71 тыс. руб., а средняя максимальная - 110,5 тыс. руб.  

4. Полная занятость является самым востребованным у работодателей типом занятости, занимая первые три места по количеству вакансий. При этом сочетание с графиком полный день - абсолютный лидер: 72% всех вакансий содержат именно такое сочетание требований. Наименее популярна у работодателей проектная работа и стажировка вкупе со сменным и гибким графиком, вахтовым методом и удаленной работой: в базе данных насчитывается лишь 97 подобных вакансий (нижние 5 строчек рейтинга), что составляет менее 1% процента всего пула вакансий.  

5. Наиболее редким требованием к опыту работы соискателей является требование свыше 6 лет - менее 3% всех вакансий его содержат. А вот самым распространенным (более половины всех вакансий) является порог от 1 года до 3 лет. На интервал от 3 до 6 лет опыта приходится чуть более четверти предложений о работе.

## Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
query_5_1 = f'''-- Сохраним запрос топ-5 компаний в новую таблицу
                WITH top AS
                (
                    SELECT
                        e.name AS employer,
                        COUNT(v.id) AS vacancies
                    FROM vacancies AS v
                    JOIN employers AS e ON v.employer_id = e.id
                    GROUP BY e.name
                    ORDER BY COUNT(v.id) DESC
                    LIMIT 5
                )

                -- Выведем первую и пятую строки из новой таблицы
                -- в порядке убывания числа вакансий
                (SELECT * FROM top LIMIT 1)
                UNION
                (SELECT * FROM top LIMIT 1 OFFSET 4)
                ORDER BY 2 DESC
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
display('Первая и пятая компания в ТОП-5 работодателей по количеству вакансий:', df)

'Первая и пятая компания в ТОП-5 работодателей по количеству вакансий:'

,employer,vacancies
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
query_5_2 = f'''WITH end_result AS
                (   -- Составим таблицу из числа работодателей в каждом регионе
                    WITH areas_employers AS
                    (
                        SELECT
                            a.*,
                            COUNT(e.id) AS num_of_employers
                        FROM public.areas AS a
                        LEFT JOIN public.employers AS e ON a.id = e.area
                        GROUP BY a.id
                        ORDER BY COUNT(e.id) DESC
                    )

                    -- К таблице с работодателями добавим количество вакансий
                    -- в каждом регионе
                    SELECT
                        a.name,
                        a.num_of_employers,
                        COUNT(v.id) AS num_of_vacancies
                    FROM areas_employers AS a
                    LEFT JOIN public.vacancies AS v ON v.area_id = a.id
                    GROUP BY a.name, a.num_of_employers
                    ORDER BY a.num_of_employers DESC
                )

                -- Выведем полученный результат, объединив с ответом на вопрос:
                -- в каком регионе нет вакансий, а число работодателей наибольшее?
                SELECT *
                FROM end_result AS e

                UNION ALL

                (SELECT
                    e.name || ' - регион без вакансий с наибольшим числом работодателей' AS name,
                    num_of_employers,
                    num_of_vacancies
                FROM end_result AS e
                WHERE num_of_vacancies = 0
                ORDER BY num_of_employers DESC
                LIMIT 1)
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
display('Рейтинг регионов по количеству работодателей и вакансий:', df)

'Рейтинг регионов по количеству работодателей и вакансий:'

,name,num_of_employers,num_of_vacancies
0,Москва,5864,5333
1,Санкт-Петербург,2217,2851
2,Минск,1115,2112
3,Алматы,721,1892
4,Екатеринбург,609,1698
...,...,...,...
1358,Кола,0,2
1359,Петропавловка (Республика Бурятия),0,1
1360,Ключи (Алтайский край),0,0
1361,Пикалево,0,1


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса
query_5_3 = f'''SELECT
                    e.name AS employer,
                    COUNT(DISTINCT v.area_id) AS num_of_areas
                FROM vacancies AS v
                JOIN employers AS e ON e.id = v.employer_id
                GROUP BY v.employer_id, e.name
                ORDER BY num_of_areas DESC
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
print('Рейтинг работодателей по количеству регионов, где они публикуют свои вакансии:', df)

Рейтинг работодателей по количеству регионов, где они публикуют свои вакансии:                                      employer  num_of_areas
0                                      Яндекс           181
1                                  Ростелеком           152
2                                  Спецремонт           116
3                      Поляков Денис Иванович            88
4                                   ООО ЕФИН             71
...                                       ...           ...
14901                            НПП Авиатрон             1
14902              Центр дистанционных торгов             1
14903  Городские Телекоммуникационные Системы             1
14904                       Введенский, Отель             1
14905              Группа Компаний Автоимпорт             1

[14906 rows x 2 columns]


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [ ]:
# текст запроса
query_5_4 = f'''SELECT
                    COUNT (*)
                FROM employers AS e
                LEFT JOIN public.employers_industries AS ei ON ei.employer_id = e.id
                WHERE ei.employer_id IS NULL
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_4,connection)
print('Количество работодателей, не указавших сферу своей деятельности:', df.iloc[0,0])

Количество работодателей, не указавших сферу своей деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [ ]:
# текст запроса
query_5_5 = f'''SELECT
                    e.name AS employer,
                    COUNT(DISTINCT ei.industry_id) AS num_of_industries
                FROM employers AS e
                LEFT JOIN employers_industries AS ei ON ei.employer_id = e.id
                WHERE ei.employer_id IS NOT NULL
                GROUP BY e.id
                HAVING COUNT(DISTINCT ei.industry_id) = 4
                ORDER BY e.name
                LIMIT 1 OFFSET 2
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
print('Третье место в алфавитном списке компаний с четырьмя сферами деятельности принадлежит:', df.iloc[0,0])

Третье место в алфавитном списке компаний с четырьмя сферами деятельности принадлежит: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса
query_5_6 = f'''SELECT
                    COUNT(e.id) AS total_it_employers
                FROM employers AS e
                LEFT JOIN employers_industries AS ei ON ei.employer_id = e.id
                JOIN public.industries AS i ON i.id = ei.industry_id
                WHERE i.name = 'Разработка программного обеспечения'
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
print('Количество работодателей, указавших "Разработку программного обеспечения" как сферу деятельности:', df.iloc[0,0])

Количество работодателей, указавших "Разработку программного обеспечения" как сферу деятельности: 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8).

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API.

In [ ]:
#  код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
links = page.find_all('a') # Ищем все ссылки на странице и сохраняем в переменной links в виде списка

# Оставим в списке только интересующие нас города
mln_city_list = [link.text for link in links[45:70]]
#mln_city_list.remove('a')

In [ ]:
# код для получения списка городов-милионников
query_5_7 = f'''WITH yandex AS
                (
                    SELECT
                        a.name AS area,
                        COUNT(v.id) AS num_of_vacancies
                    FROM vacancies AS v
                    JOIN employers AS e ON e.id = v.employer_id
                    JOIN areas AS a ON a.id = v.area_id
                    WHERE e.name = 'Яндекс' AND a.name IN {tuple(mln_city_list)}
                    GROUP BY a.name
                    ORDER BY num_of_vacancies
                )

                SELECT * FROM yandex
                UNION ALL
                SELECT 'Total', SUM(ya.num_of_vacancies)::int FROM yandex AS ya
             '''

In [ ]:
# текст запроса
df = pd.read_sql_query(query_5_7, connection)
display('Количество вакансий "Яндекса" в городах-миллионниках:', df)

'Количество вакансий "Яндекса" в городах-миллионниках:'

,area,num_of_vacancies
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


In [ ]:
# результат запроса

***

***Выводы по анализу работодателей:***  
1. Пятерку работодателей по количеству вакансий открывает Яндекс, чье количество предложений о работе - 1933 - почти в полтора раза превышает сумму вакансий у остальных членов ТОП-5 (для которых медианой является 444 вакансии). Замыкает пятерку Газпром нефть с 331 открытой позицией.  

2. Как и в случае с количеством вакансий в предыдущем разделе исследования, тройку городов-лидеров по числу работодателей представляют Москва, Санкт-Петербург и Минск. А регионом, где вакансии отсутствуют, но зарегистрировано наибольшее число работодателей (410) стал регион Россия.  

3. Наиболее широкой географией по поиску сотрудников могут похвастаться Яндекс, Ростелеком и Спецремонт. Их вакансии представлены в 181, 152 и 116 регионах соответственно. Примечательно, что в отличии от первых двух компаний, входящих в ТОП-5 по числу вакансий, Спецремонт находится по этому показателю лишь на 15 месте (у компании всего 151 открытая позиция). Таким образом, Спецремонт при относительно небольшом количестве вакансий охватывает довольно значительную территорию.  

4. Из всех компаний, представленных в базе данных (23,5 тыс.) более трети (8,4 тыс.) не указали сферу своей деятельности, что может негативно повлиять на поиск их вакансий потенциальными работниками.  

5. Максимальное количество сфер деятельности, в которых представленные компании ведут свой бизнес - 5 (к этой группе отноится Тинькофф), медианное значение этого показателя - 2. В список компаний с 4-мя сферами бизнеса вошла компания 2ГИС, принадлежащая Сбербанку. При этом у самого СБЕРа лишь одна сфера деятельности.  

6. Разработкой программного обеспечения занимаются 15% всех работодателей (3553 компании). Однако вполне возможно, что количество таких компаний в действительности больше, т.к. не все работодатели указали сферу свой деятельности.  

7. Четверть всех вакансий (485 из 1933) лидера по этому показателю - Яндекса - приходится на города-миллионники России. При этом медианное количество открытых позиций в этих городах - 26, а среднее (из-за вхождения Москвы, Питера и нескольких других крупных городов) - 30. Это указывает на  то, что география привлечения сотрудников Яндексом очень широкая, а количество вакансий (и, вероятно, их разноплановость) этой географией определяются в незначительной степени.  

## Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [ ]:
# текст запроса
query_6_1 = f'''SELECT
                    COUNT(id)
                FROM vacancies
                WHERE lower(name) LIKE '%data%' OR lower(name) LIKE '%данн%'
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
print('Количество вакансий, имеющих отношение к данным:', df.iloc[0,0])

Количество вакансий, имеющих отношение к данным: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста?
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.


In [ ]:
# текст запроса
query_6_2 = f'''SELECT
                    COUNT(id)
                FROM vacancies
                WHERE (lower(name) LIKE '%data scientist%'
                    OR lower(name) LIKE '%data science%'
                    OR lower(name) LIKE '%исследователь данных%'
                    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                    OR lower(name) LIKE '%machine learning%'
                    OR lower(name) LIKE '%машинн%обучен%')
                    AND (lower(name) LIKE '%junior%'
                    OR experience = 'Нет опыта'
                    OR employment = 'Стажировка')
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
print('Количество вакансий для начинающих дата-сайентистов:', df.iloc[0,0])

Количество вакансий для начинающих дата-сайентистов: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_6_3 = f'''SELECT
                    COUNT(id)
                FROM vacancies
                WHERE (lower(name) LIKE '%data scientist%'
                    OR lower(name) LIKE '%data science%'
                    OR lower(name) LIKE '%исследователь данных%'
                    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                    OR lower(name) LIKE '%machine learning%'
                    OR lower(name) LIKE '%машинн%обучен%')
                    AND (key_skills LIKE '%SQL%'
                    OR key_skills LIKE '%postgres%')
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
print('Количество вакансий, в которых SQL или postgres указаны в качестве ключевых навыков:', df.iloc[0,0])

Количество вакансий, в которых SQL или postgres указаны в качестве ключевых навыков: 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = f'''SELECT
                    COUNT(id)
                FROM vacancies
                WHERE (lower(name) LIKE '%data scientist%'
                    OR lower(name) LIKE '%data science%'
                    OR lower(name) LIKE '%исследователь данных%'
                    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                    OR lower(name) LIKE '%machine learning%'
                    OR lower(name) LIKE '%машинн%обучен%')
                    AND (key_skills LIKE '%Python%')
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
print('Количество вакансий, в которых Python указан в качестве ключевого навыка:', df.iloc[0,0])

Количество вакансий, в которых Python указан в качестве ключевого навыка: 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса
query_6_5 = f'''SELECT
                    -- Для расчета количества навыков, отделенных символом
                    -- табуляции, вычтем из исходной строки строку с
                    -- удаленными знаками табуляции (и добавим единицу),
                    -- округлим до двух знаков после запятой.
                    ROUND(AVG(
                        char_length(key_skills)
                        - char_length(replace(key_skills, '\t', '')) + 1
                    ),2) AS avg_num_of_skills
                FROM vacancies
                WHERE (lower(name) LIKE '%data scientist%'
                    OR lower(name) LIKE '%data science%'
                    OR lower(name) LIKE '%исследователь данных%'
                    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                    OR lower(name) LIKE '%machine learning%'
                    OR lower(name) LIKE '%машинн%обучен%')
                    AND key_skills IS NOT NULL
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
print('Среднее количество навыков в вакансиях для DS:', df.iloc[0,0])

Среднее количество навыков в вакансиях для DS: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*).

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [ ]:
# текст запроса
query_6_6 = f'''SELECT DISTINCT
                    experience,
                    ROUND(AVG(
                        (coalesce(salary_from, salary_to)   -- Если не указана начальная зарплата, то подставим максимальную
                        + coalesce(salary_to, salary_from)) -- Если не указана максимальная, то подставим начальную
                        /2),0)::int AS avg_salary           -- Тогда при делении на 2, получим ее же; округлим, отбросим десятичную часть
                FROM vacancies
                WHERE (lower(name) LIKE '%data scientist%'
                    OR lower(name) LIKE '%data science%'
                    OR lower(name) LIKE '%исследователь данных%'
                    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                    OR lower(name) LIKE '%machine learning%'
                    OR lower(name) LIKE '%машинн%обучен%')
                    AND (salary_from IS NOT NULL
                        OR salary_to IS NOT NULL)
                GROUP BY experience
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
display('Средние зарплаты дата-сайентистов в зависимости от опыта работы:', df)

'Средние зарплаты дата-сайентистов в зависимости от опыта работы:'

,experience,avg_salary
0,Нет опыта,74643
1,От 1 года до 3 лет,139675
2,От 3 до 6 лет,243115


***

### Bыводы по предметному анализу
В исследуемой базе данных содержится 1771 вакансия, имеющая отношения к анализу данных, что составляет 3,5% от общего количества вакансий.

И лишь около 3% (или 51 вакансия) из этого числа подходят для начинающих дата-сайентистов, что в масштабе всей базы вакансий не превышает и 1%! Таким образом, несмотря на популярность области data science, начинающему специалисту довольно сложно найти первую работу.

Владение SQL или PostgreSQL ожидается от потенциальных сотрудников в 201 вакансии. Представляя собой лишь 11% всех DS-вакансий, это число указывает на то, что у этих работодателей данные уже собраны в файлах типа .csv или что в штате есть профильные специалисты, готовые по требованию сформировать запрос к базе данных. Другим объяснением может быть то, что от дата сайентиста ожидается, что он будет собирать данные главным образом с помощью парсинга сайтов, либо заниматься искллючительно разработкой моделей машинного обучения на готовых данных. Кроме этого, без учета требования к стажу, можно предположить, что на позициях ведущего дата-сайентиста (senior или lead) компании меньше всего ожидают от своих сотрудников умения составлять SQL-запросы. В любом случае, даже базовые знания SQL (которые могут никогда не пригодится в работе) позволят кандидату указать этот навык в своем резюме, что при прочих равных поднимет его в глазах работодателя по сравнению с другими соискателями.

Знание Python является ключевым навыком в чуть большем количестве DS-вакансий (351) и представляет собой почти 20% от всего числа профильных. Интересно отметить, что в половине этих вакансий другим ключевым навыком также указан SQL или PostgreSQL, а в другой половине не указан ни один. Можно заключить, что связка Python-SQL является чрезвычайно популярной у работодателей. А вот если в предыдущем пункте исследования ужесточить требования и отбросить навыки программирования на таких языках, как Python, C++ и Java, то вакансий с ключевыми навыками SQL или PostgreSQL окажется лишь 25 (1,5%). Напрашивается очевидный вывод: работа в Data Science области возможна без знания SQL, но вряд ли возможна без знания языков программирования.

С учетом того, что среднее число навыков в вакансиях по DS равно 6.41, то указание в своем резюме лишь двух (Python и SQL) позволяет кандидату на треть соотвествовать требованиям работодателя (без оценки качества и глубины этих знаний). А при получении последующих знаний и навыков работы с моделями машинного обучения соискатель значительно повысит свои шансы в глазах работодателя, тем более если работодатель использует программные алгоритмы отбора резюме.

Несмотря на низкий процент вакансий для начинающих дата-сайентистов, их средняя заработная плата может составлять почти 75 тыс. руб. При этом лишь спустя год работы специалист может рассчитывать на увеличение зарплаты на 86% - почти до 140 тыс., а спустя еще 2 года работы - на 74%, до 243 тыс. (в среднем по всем проанализированым вакансиям). Таким образом, область DS предлагает не только достойное вознаграждение на старте, но и быстрое продвижение по карьерной лестнице.

## Юнит 7. Дополнительный анализ: сферы деятельности

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных,
# сделать прогнозы, продумать варианты продолжения исследования

1. Дополним юнит предварительного анализа и узнаем, какое количество работодателей в настоящее время размещает вакансии.

In [ ]:
query_7_1 = f'''SELECT
                    COUNT(DISTINCT employer_id) AS employers_with_vacancies
                FROM vacancies
             '''

In [ ]:
df = pd.read_sql_query(query_7_1, connection)
print(f'Все вакансии в базе данных относятся к {df.iloc[0,0]} работодателям.')

Все вакансии в базе данных относятся к 14906 работодателям.


2. Посмотрим, в каких сферах деятельности представлено больше всего работодателей, а также вакансий. Отсортируем список по количеству вакансий в порядке убывания.

In [ ]:
query_7_2 = f'''SELECT
                    i.name AS industry,
                    COUNT(DISTINCT ei.employer_id) AS num_of_employers,
                    COUNT(v.id) AS num_of_vacancies
                FROM industries AS i
                LEFT JOIN employers_industries AS ei ON ei.industry_id = i.id
                LEFT JOIN employers AS e ON e.id = ei.employer_id
                LEFT JOIN vacancies AS v ON v.employer_id = e.id
                GROUP BY i.name
                ORDER BY COUNT(v.id) DESC
             '''

In [ ]:
df = pd.read_sql_query(query_7_2, connection)
display('Рейтинг сфер деятельности по количеству работодателей и вакансий:', df)

'Рейтинг сфер деятельности по количеству работодателей и вакансий:'

,industry,num_of_employers,num_of_vacancies
0,Разработка программного обеспечения,3553,12499
1,"Системная интеграция, автоматизации технологи...",2993,11034
2,"Интернет-компания (поисковики, платежные систе...",1675,6413
3,Банк,249,2742
4,Мобильная связь,156,1585
...,...,...,...
289,Химчистка,7,4
290,"Ассоциация в сфере культуры, искусства",1,2
291,Автошкола,3,1
292,Ритуальные услуги,4,1


3. Узнаем, в каких сферах деятельности больше всего вакансий для начинаюших специалистов, т.е. не имеющих опыта работы. Выведем ТОП-10.

In [ ]:
query_7_3 = f'''SELECT
                    i.name AS industry,
                    COUNT(DISTINCT v.id) AS num_of_vacancies
                FROM industries AS i
                LEFT JOIN employers_industries AS ei ON ei.industry_id = i.id
                LEFT JOIN employers AS e ON e.id = ei.employer_id
                LEFT JOIN vacancies AS v ON v.employer_id = e.id
                WHERE v.experience = 'Нет опыта'
                GROUP BY i.name
                ORDER BY COUNT(v.id) DESC
                LIMIT 10
             '''

In [ ]:
df = pd.read_sql_query(query_7_3, connection)
display('ТОП-10 сфер деятельности по количеству вакансий без опыта работы:', df)

'ТОП-10 сфер деятельности по количеству вакансий без опыта работы:'

,industry,num_of_vacancies
0,"Интернет-компания (поисковики, платежные систе...",1930
1,Разработка программного обеспечения,1225
2,"Системная интеграция, автоматизации технологи...",1217
3,Мобильная связь,369
4,Интернет-провайдер,320
5,Оптоволоконная связь,303
6,Фиксированная связь,286
7,Банк,191
8,Консалтинговые услуги,186
9,"Маркетинговые, рекламные, BTL, дизайнерские, E...",126


4. Проверим, к каким сферам относятся вакансии, связанные с данными и Data Science. Выведем ТОП-5 по убыванию количества вакансий.

In [ ]:
query_7_4 = f'''SELECT
                    i.name AS industry,
                    COUNT(DISTINCT v.id) AS num_of_vacancies
                FROM industries AS i
                LEFT JOIN employers_industries AS ei ON ei.industry_id = i.id
                LEFT JOIN employers AS e ON e.id = ei.employer_id
                LEFT JOIN vacancies AS v ON v.employer_id = e.id
                WHERE (lower(v.name) LIKE '%data%'
                    OR lower(v.name) LIKE '%данн%'
                    OR lower(v.name) LIKE '%data scientist%'
                    OR lower(v.name) LIKE '%data science%'
                    OR lower(v.name) LIKE '%исследователь данных%'
                    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
                    OR lower(v.name) LIKE '%machine learning%'
                    OR lower(v.name) LIKE '%машинн%обучен%')
                GROUP BY i.name
                ORDER BY COUNT(v.id) DESC
                LIMIT 5
             '''

In [ ]:
df = pd.read_sql_query(query_7_4, connection)
display('ТОП-5 сфер деятельности по количеству DS-вакансий:', df)

'ТОП-5 сфер деятельности по количеству DS-вакансий:'

,industry,num_of_vacancies
0,Разработка программного обеспечения,556
1,"Системная интеграция, автоматизации технологи...",470
2,Банк,308
3,"Интернет-компания (поисковики, платежные систе...",243
4,Мобильная связь,145


5. Есть мнение, что зарплаты в Москве выше, чем в других городах. Проверим, так ли это. Для этого возьмем разницу в процентах между начальной зарплатой в Москве и других городах и вычислим среднее по каждой вакансии. Условимся, что для данного исследования названия вакансий, график работы, тип занятости и опыт работы должны совпадать.

In [ ]:
query_7_5 = f'''SELECT
                    a.name AS vacancy,
                    -- Вычислим среднюю разницу между зарплатой в Москве
                    -- и других городах в процентах
                    ROUND(
                        AVG(a.salary_from * 100
                            / b.salary_from - 100)
                        ,0) AS "salary_difference,%"
                FROM vacancies AS a
                -- Добавим таблицу вакансий по нескольким столбцам,
                -- чтобы обеспечить максимальное сходство вакансий
                JOIN vacancies AS b ON a.name = b.name
                    AND a.schedule = b.schedule
                    AND a.experience = b.experience
                    AND a.employment = b.employment
                LEFT JOIN areas AS ar_a ON a.area_id = ar_a.id
                LEFT JOIN areas AS ar_b ON b.area_id = ar_b.id
                WHERE (a.salary_from IS NOT NULL
                    OR b.salary_from IS NOT NULL)
                    AND a.salary_from != b.salary_from
                    AND (ar_a.name = 'Москва' AND ar_b.name != 'Москва')
                GROUP BY a.name
                ORDER BY "salary_difference,%" DESC
             '''

In [ ]:
df = pd.read_sql_query(query_7_5, connection)
display('Разница в зарплатах по одинаковым вакансиям с Москвой (в процентах):', df)

'Разница в зарплатах по одинаковым вакансиям с Москвой (в процентах):'

,vacancy,"salary_difference,%"
0,Аналитик 1С,11223.0
1,Программист Python,10246.0
2,Менеджер по продажам,589.0
3,Консультант по информационным системам,555.0
4,Системный администратор,529.0
...,...,...
129,Программист 1С (удаленно),-33.0
130,Backend Разработчик,-34.0
131,Data Scientist,-50.0
132,QA Automation Engineer,-50.0


6. В завершении дополним анализ вакансий, узнав, в каких странах, кроме России, они еще предлагаются. Воспользуемся списком 193 стран-членов ООН [отсюда](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%81%D1%83%D0%B4%D0%B0%D1%80%D1%81%D1%82%D0%B2).

In [ ]:
# Найдем и выгрузим список стран со страницы
url = 'https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%81%D1%83%D0%B4%D0%B0%D1%80%D1%81%D1%82%D0%B2'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
links = page.find_all('a') # Ищем все ссылки на странице и сохраняем в переменной links в виде списка

# Удаляем все ненужное из списка
country_list = [link.text for link in links[35:420]]
country_list = [country for country in country_list if country != '']

In [ ]:
query_7_6 = f'''-- Созданим таблицу с вакансиями во всех странах
                WITH detailed AS
                (
                    SELECT
                        a.name AS country,
                        COUNT(v.id) AS num_of_vacancies
                    FROM vacancies AS v
                    LEFT JOIN areas AS a ON a.id = v.area_id
                    WHERE a.name IN {tuple(country_list)}
                    GROUP BY a.name
                    ORDER BY num_of_vacancies DESC
                )

                -- Выведем список всех стран
                SELECT * FROM detailed

                -- Нижеследующая часть запроса подходит, когда не нужно
                -- строить график со всеми странами, и можно страны
                -- с небольшим вкладом (например, меньше 10 вакансий)
                -- объединить в категорию "Прочие страны".
                -- От этой части пришлось отказаться ради построения
                -- графика.
                -- SELECT *
                -- FROM detailed
                -- WHERE num_of_vacancies >= 10
                -- UNION ALL
                -- SELECT
                    --'Прочие страны',
                    --SUM(num_of_vacancies)::int
                -- FROM detailed
                -- WHERE num_of_vacancies < 10
             '''

In [ ]:
df = pd.read_sql_query(query_7_6, connection)
display('Количество вакансий за пределами России:', df)

'Количество вакансий за пределами России:'

,country,num_of_vacancies
0,Армения,223
1,Кипр,136
2,Польша,53
3,Турция,39
4,Сербия,26
5,Литва,19
6,Германия,19
7,Черногория,14
8,Чехия,14
9,ОАЭ,12


***Посмотрим страны на карте мира***

In [ ]:
# Для этого добавим столбец с переводом названий стран на английский язык методами Python
eng_list = ['Armenia', 'Cyprus', 'Poland', 'Turkey', 'Serbia', 'Lithuania',
            'Germany', 'Montenegro', 'Czechia', 'United Arab Emirates',
            'United States of America', 'Bulgaria', 'Latvia', 'Hungary',
            'Finland', 'Spain', 'Tajikistan', 'Austria', 'Estonia', 'Israel',
            'Netherlands', 'United Kingdom of Great Britain and Northern Ireland',
            'India', 'China', 'Romania', 'Indonesia', 'Slovakia']
df['country_eng'] = df['country']
for i,country in enumerate(eng_list):
    df.iloc[i, 2] = country # iloc

# Построим тепловую картограмму
fig = px.choropleth(
    data_frame=df, # исходный dataframe
    locations=df['country_eng'], # список стран
    locationmode='country names', # режим сопоставления локаций с базой Plotly
    color='num_of_vacancies', # зависимость цветовой заливки от количества вакансий
    range_color=[0, 223], # диапазон изменения цветовой гаммы
    title='Количество вакансий за пределами России', # название графика
    labels={'num_of_vacancies': 'Вакансии', 'country_eng': 'Страна'}, # названия меток
    color_continuous_scale='deep', # цветовая схема
    projection='miller', # вид проекции Земли
    scope='world', # регион Земли для представления на графике - весь мир
    fitbounds='locations', # рамки графика - вмещает рассматриваемые страны
    width=800, # ширина графика
    height=600 # высота графика
)

fig.show();
# fig.write_html("plotly/world_map.html")

[![Количество вакансий за пределами России](plotly/world_map.png 'Нажмите, чтобы перейти к интерактивному графику')]
(http://htmlpreview.github.io/?https://github.com/Ivan-Pankov/DataScience_SF/blob/master/Project_2_Job_Analysis_HH/plotly/world_map.html)

In [ ]:
# Закрываем подключение к базе данных
connection.close()

**Выводы по дополнительному анализу сфер деятельности:**  
1. Оказывается, все имеющиеся в базе данных вакансии (49197) принадлежат 63% компаний (14906 из 23501). Иначе говоря, более трети работодателей числятся в базе, но в настоящее время не набирают сотрудников. К тому же, как мы видели ранее, не все компании указали сферу своей деятельности. Поэтому, если мы хотим понять, в каких сферах больше всего шансов отыскать работу, следует оценить их не только с точки зрения количества работодателей, но и в разрезе количества открытых вакансий.  

2. Несмотря на сортировку по вакансиям, первые три сферы деятельности одинаково успешно держат первенство и по количеству работодателей. Банки и компании мобильной связи замыкают ТОП-5 по числу вакансий, что вполне очевидно, т.к. эти сферы активно развиваются и нуждаются в обработке и анализе больших объемов данных о клиентах. Кроме этого, в предыдущем исследовании *ТОП-5 работодателей* в число лидеров вошли банки Тинькофф и СБЕР, а также коммуникационный оператор Ростелеком, чем и объясняется в данном случае высокий рейтинг связанных с ними сфер деятельности.
    
    Сразу за ТОП-5 сфер наблюдается резкое снижение как количества работодателей, так и количества открытых вакансий, что указывает на исключительную популярность и стремительное развитие первых сфер деятельности. Отметим, что сферой, в которой представлен лишь один работодатель, является "Ассоциация в сфере культуры, искусства", а сфера, где нет ни одной вакансии (хотя есть три работодателя) - "Ботанический сад, зоопарк, заповедник".

    При этом не стоит забывать, что *ТОП-5 работодателей* по количеству открытых вакасий замыкает "Газпром нефть". Однако сферы деятельности этой компании ("Нефтепереработка, нефтехимия (производство)", "ГСМ, топливо (продвижение, оптовая торговля)" и "ГСМ, топливо (розничная торговля)") не вошли даже в десятку нашего рейтинга. Это означает, что сфера деятельности не обязательно должна насчитывать большое количество работодателей, чтобы предлагать соискателям массу вакансий, т.к. один крупный работодатель может предложить много свободных позиций. Яркий пример этому - компания "Яндекс".  

3. Как ни странно, но все те же сферы деятельности предлагают наибольшее число вакансий для начинающих специалистов без опыта работы. Это подтверждает вывод о том, что данные сферы претерпевают бурный рост, и компании, остро нуждающиеся в работниках, готовы "выращивать" специалистов внутри с прицелом на будущее. Отметим, что и в остальных категориях опыта работы лидерами являются эти же сферы деятельности.  

4. Ожидаемо, наибольшее количество вакансий, связанных с данными и Data Science обнаруживается в рассмотренных выше быстро развивающихся сферах. Именно в них требуется работа с большими данными, а также моделирование и предсказание поведения клиентов. Однако ими все не ограничивается. Полный рейтинг насчитывает 195 сфер деятельности (66% от полного списка), где востребована наука о данных, и к ним относятся розничные сети по продаже товаров, складские услуги, водоснабжение и канализация и даже ветеринарная деятельность!  

5. В целом, представление о том, что зарплаты в Москве выше, чем в других городах, подтвердилось. За исключением двух аномалий в полученной выгрузке, где разница превышает 10 тыс. процентов (ошибка в нескольких порядках при указании зарплаты в других городах), которыми нужно пренебречь, видно, что одна и та же позиция в столице может оплачиваться во много раз выше. А вот меньший размер оплаты труда по схожим вакансиям редко превышает 50% по сравнению с Москвой, и таких вакансий гораздо меньше.  

    К тому же, в случаях более высоких московских зарплат чаще встречаются оффлайн-вакансии, такие как: Администратор проектов, Ведущий менеджер по продажам, - в то время, как более низкие московские зарплаты практически полностью характерны для онлайн-позиций. Последнее наблюдение вполне объяснимо: если сотрудник может работать на московской позиции удаленно из другого города, где жизнь дешевле, то зарплату можно снизить, тогда как офисному работнику в самой Москве приходится увеличивать жалование ввиду высокой стоимости жизни.  

    Еще раз отметим, что для данного исследования были сделаны оговороки, могущие влиять на результат: вакансии сгруппированы по названию, но не по работодателю, не учтен верхний порог зарплатной вилки, отброшены вакансии, в которых работодатель не указал размер зарплаты, а также такие вакансии, где зарплата в разных городах совпадает. Впрочем, последних насчитывается лишь 81, что намного меньше, чем 134 случая различия в зарплатах. Кроме этого разница иногда расчитывалась лишь на основании одной вакансии, что является не слишком репрезентативным.

6. За пределами России работодатели предлагают 632 позиции, что составляет чуть более 1% всей базы вакансий. Это понятно, т.к. портал hh.ru ориентирован на работодателей из постсоветского пространства. Геогра

# Общий вывод по проекту

1. Первым делом стоит сказать, что для точных оценок и прогнозов и тем более моделирования использованных данных из базы недостаточно: 49 тыс. вакансий, 23,5 тыс. работодателей, треть из которых не указали свою сферу деятельности, и треть из которых не размещает вакансии. Для получения результатов, на которые можно опираться в дальнейшей работе, желательно увеличить объем входных данных минимум вдвое (100 тыс. вакансий).  

2. Тем не менее, сделанные выводы позволяют сформировать общее представление о положении дел в рекрутинговой сфере. Так, например, при средней минимальной зарплате по всем вакансиям в 71 тыс. начинающий дата-сайентист может рассчитывать на 75 тыс. (в среднем) причем вне зависимости от региона и сфера деятельности.  

3. Впрочем, только 3,5% (1771) всех вакансий имеют отношение к анализу данных и лишь в 3% из них (51 вакансия) работодатели готовы взять дата-сайентиста без опыта или на стажировку.  

4. Однако поскольку сферы деятельности, связанные с разработкой ПО, автоматизацией, интернет, а также банками и мобильной связью претерпевают бурный рост, они стремительно расширяются (это подтверждается количеством открытых вакансий, охваченных регионов, а также числом самих компаний) и остро нуждаются в специалистах: в больше половине вакансий требуется опыт работы от 1 до 3 лет; еще четверть приходится на специалистов с опытом от 3 до 6 лет.  

5. И хотя вакансий без опыта работы во всех сферах насчитывается не более 15%, можно ожидать увеличения их числа (в первую очередь за счет перечисленных выше сфер), а также хорошие карьерные перспективы для вступивших на этот путь. Например, дата-сайентисты, проработавшие год, могут рассчитывать на удвоение своей зарплаты, а еще спустя несколько лет - на очередное удвоение!  

6. Такое развитие обеспечивается крупнейшими игроками в своих сферах деятельности: Яндексом, Ростелекомом, Газпром нефтью, банками Тинькофф и СБЕР. Именно они активно привлекают и взращивают специалистов XXI века и делают это не только в крупных городах, таких как: Москва, Санкт-Петербург, Минск, Алматы, Екатеринбург, - но и в различных регионах страны. Например, лишь четверть вакансия Яндекса, как мы видели, приходится на города-миллионники.  

7. От желающих связать свою карьеру с цифровыми технологиями работодатели ожидают познания в различных областях: в среднем от соискателя требуется свыше 6 различных навыков. Однако обилие информации и ее доступность позволяют относительно легко удовлетворять этим требованиям. Так, владение SQL вкупе с одним из популярных языков программирования (Python, Java, C++) сразу “закрывают” два ключевых навыка и позволяют откликаться почти на четверть вакансий, связанных с анализом данных.  

8. Единственное, на что не стоит рассчитывать соискателям уровня junior - это на работу в шезлонге на берегу океана. Лишь 16% всех вакансий предлагают удаленную работу (и в первую очередь опытным специалистам) и чуть более 1% вакансий находятся за пределами России. 72% работодателей все еще ожидают видеть своих сотрудников в офисе полный рабочий день. Однако можно прогнозировать послабление этих требований после опыта пандемии и в связи с необходимостью сокращать расходы.  

9. В 200 сферах деятельности из проанализированных 294 (что составляет 68%) наблюдается потребность в дата-сайентистах. 3,5 тыс. компаний из базы данных заняты разработкой программного обеспечения. Пока их число составляет 15%, но после первых обещающих результатов анализа больших данных, их использования в прогностических моделях и влияния на бизнес, следует ожидать увеличения этого числа в самом ближайшем будущем.